<a href="https://colab.research.google.com/github/Carlos1729/DGL/blob/main/SimpleGCN_OGBN_Arxiv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.2 MB/s eta 0:00:00


In [2]:
!pip install  dgl -f https://data.dgl.ai/wheels/cu116/repo.html


Looking in links: https://data.dgl.ai/wheels/cu116/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 1.9 MB/s eta 0:00:00


In [3]:
!pip install  dglgo -f https://data.dgl.ai/wheels-test/repo.html


Looking in links: https://data.dgl.ai/wheels-test/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 966.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 51.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7029 sha256=ad6ee3e71056b3900c4a150f78826c45679e9e29cd706c579362bddde75f4c3b
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463c

In [4]:
!pip install torchmetrics==0.11.4


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 8.2 MB/s eta 0:00:00


In [5]:
import math
import time

import dgl
import dgl.function as fn

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.data import (
    CiteseerGraphDataset,
    CoraGraphDataset,
    PubmedGraphDataset,
    register_data_args,
)
from dgl.nn.pytorch.conv import SGConv

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [6]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)[mask]  # only compute the evaluation set
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [27]:
from ogb.nodeproppred import DglNodePropPredDataset

In [28]:
data = dgl.data.AsNodePredDataset(DglNodePropPredDataset("ogbn-arxiv"))

Downloaded 0.08 GB: 100%|██████████| 81/81 [00:08<00:00,  9.29it/s]


Extracting dataset/arxiv.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 1818.87it/s]


Converting graphs into DGL objects...


100%|██████████| 1/1 [00:00<00:00, 61.67it/s]

Saving...


In [29]:
g = data[0]
g = g.int().to("cuda")

In [31]:
data

Dataset("ogbn-arxiv-as-nodepred", num_graphs=1, save_path=/root/.dgl/ogbn-arxiv-as-nodepred)

In [32]:
features = g.ndata["feat"]
labels = g.ndata["label"]
train_mask = g.ndata["train_mask"]
val_mask = g.ndata["val_mask"]
test_mask = g.ndata["test_mask"]
in_feats = features.shape[1]
n_classes = data.num_classes
n_edges = g.num_edges()

In [33]:
print(
        """----Data statistics------'
      #Edges %d
      #Classes %d
      #Train samples %d
      #Val samples %d
      #Test samples %d"""
        % (
            n_edges,
            n_classes,
            train_mask.int().sum().item(),
            val_mask.int().sum().item(),
            test_mask.int().sum().item(),
        )
    )

----Data statistics------'
      #Edges 1166243
      #Classes 40
      #Train samples 90941
      #Val samples 29799
      #Test samples 48603


In [34]:
train_mask

tensor([ True,  True,  True,  ..., False, False, False], device='cuda:0')

In [35]:
val_mask

tensor([False, False, False,  ..., False, False, False], device='cuda:0')

In [36]:
test_mask

tensor([False, False, False,  ...,  True,  True,  True], device='cuda:0')

In [37]:
n_edges = g.num_edges()
# add self loop
g = dgl.remove_self_loop(g)
g = dgl.add_self_loop(g)

# create SGC model
model = SGConv(in_feats, n_classes, k=2, cached=True, bias=False)
model.cuda()

SGConv(
  (fc): Linear(in_features=128, out_features=40, bias=False)
)

In [39]:
loss_fcn = torch.nn.CrossEntropyLoss()

# use optimizer
optimizer = torch.optim.Adam(
    model.parameters(), lr=0.2, weight_decay=5e-6
)

# initialize graph
dur = []

In [40]:
for epoch in range(100):
    print(epoch)
    model.train()
    if epoch >= 3:
        t0 = time.time()
    # forward
    logits = model(g, features)  # only compute the train set
    loss = loss_fcn(logits[train_mask], labels[train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >= 3:
        dur.append(time.time() - t0)

    acc = evaluate(model, g, features, labels, val_mask)
    print(
        "Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | Accuracy {:.4f} | "
        "ETputs(KTEPS) {:.2f}".format(
            epoch,
            np.mean(dur),
            loss.item(),
            acc,
            n_edges / np.mean(dur) / 1000,
        )
    )

print()
acc = evaluate(model, g, features, labels, test_mask)
print("Test Accuracy {:.4f}".format(acc))


0
Epoch 00000 | Time(s) nan | Loss 3.8674 | Accuracy 0.1750 | ETputs(KTEPS) nan
1
Epoch 00001 | Time(s) nan | Loss 5.0450 | Accuracy 0.2166 | ETputs(KTEPS) nan
2
Epoch 00002 | Time(s) nan | Loss 6.9968 | Accuracy 0.2555 | ETputs(KTEPS) nan
3
Epoch 00003 | Time(s) 0.0097 | Loss 5.9077 | Accuracy 0.2721 | ETputs(KTEPS) 119856.36
4
Epoch 00004 | Time(s) 0.0097 | Loss 4.9246 | Accuracy 0.1434 | ETputs(KTEPS) 119976.89
5
Epoch 00005 | Time(s) 0.0097 | Loss 5.4874 | Accuracy 0.1390 | ETputs(KTEPS) 119708.72
6
Epoch 00006 | Time(s) 0.0098 | Loss 5.6600 | Accuracy 0.2706 | ETputs(KTEPS) 119601.40
7
Epoch 00007 | Time(s) 0.0096 | Loss 5.0532 | Accuracy 0.3517 | ETputs(KTEPS) 121191.45
8
Epoch 00008 | Time(s) 0.0095 | Loss 5.1905 | Accuracy 0.3589 | ETputs(KTEPS) 122787.76
9
Epoch 00009 | Time(s) 0.0095 | Loss 4.8489 | Accuracy 0.3672 | ETputs(KTEPS) 122792.89
10
Epoch 00010 | Time(s) 0.0094 | Loss 4.2529 | Accuracy 0.3221 | ETputs(KTEPS) 123651.15
11
Epoch 00011 | Time(s) 0.0094 | Loss 3.7454 |

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00013 | Time(s) 0.0094 | Loss 3.6281 | Accuracy 0.3732 | ETputs(KTEPS) 123863.92
14
Epoch 00014 | Time(s) 0.0094 | Loss 3.4246 | Accuracy 0.3977 | ETputs(KTEPS) 123787.27
15
Epoch 00015 | Time(s) 0.0095 | Loss 3.2698 | Accuracy 0.4284 | ETputs(KTEPS) 122687.23
16
Epoch 00016 | Time(s) 0.0096 | Loss 3.0911 | Accuracy 0.4370 | ETputs(KTEPS) 121096.21
17
Epoch 00017 | Time(s) 0.0096 | Loss 3.0863 | Accuracy 0.4494 | ETputs(KTEPS) 121699.60
18
Epoch 00018 | Time(s) 0.0095 | Loss 2.9196 | Accuracy 0.4659 | ETputs(KTEPS) 122198.94
19
Epoch 00019 | Time(s) 0.0095 | Loss 2.6287 | Accuracy 0.4902 | ETputs(KTEPS) 122520.61
20
Epoch 00020 | Time(s) 0.0095 | Loss 2.3578 | Accuracy 0.4765 | ETputs(KTEPS) 122999.25
21
Epoch 00021 | Time(s) 0.0094 | Loss 2.2978 | Accuracy 0.4245 | ETputs(KTEPS) 123462.01
22
Epoch 00022 | Time(s) 0.0094 | Loss 2.3953 | Accuracy 0.3854 | ETputs(KTEPS) 123868.61
23
Epoch 00023 | Time(s) 0.0093 | Loss 2.4012 | Accuracy 0.4099 | ETputs(KTEPS) 125658.59
24
Epoch 0002